In [20]:
import pandas as pd
from paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location,pickup_longitude,pickup_latitude,target_rides_next_hour
0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,3.0,1.0,...,0.0,3.0,1.0,1.0,0.0,2024-01-29,Albany Park,-87.721559,41.968069,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,1.0,3.0,1.0,2.0,1.0,2024-01-30,Albany Park,-87.721559,41.968069,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,...,3.0,0.0,2.0,0.0,0.0,2024-01-31,Albany Park,-87.721559,41.968069,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,1.0,...,0.0,2.0,1.0,0.0,0.0,2024-02-01,Albany Park,-87.721559,41.968069,0.0
4,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,0.0,...,0.0,2.0,1.0,4.0,1.0,2024-02-02,Albany Park,-87.721559,41.968069,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2025-01-27,Andersonville,-87.671446,41.979796,0.0
35051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2025-01-28,Andersonville,-87.671446,41.979796,0.0
35052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2025-01-29,Andersonville,-87.671446,41.979796,0.0
35053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2025-01-30,Andersonville,-87.671446,41.979796,0.0


In [21]:
from datetime import datetime
from data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2024, 11, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(26315, 676)
y_train.shape=(26315,)
X_test.shape=(8740, 676)
y_test.shape=(8740,)


In [22]:
import lightgbm as lgb

In [23]:
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [24]:
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.076327 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123842
[LightGBM] [Info] Number of data points in the train set: 26315, number of used features: 672
[LightGBM] [Info] Start training from score 4.649933


LGBMRegressor()

In [25]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)

from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=1.2607


In [26]:
from plot import plot_one_sample

plot_one_sample(
    example_id='Loop',
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)